## Download Gaia Data

In [ ]:
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

In [ ]:
MAGNITUDE_LIMIT = 8.0
GAIA_PATH = f'../data/gaia_m<{MAGNITUDE_LIMIT}.ecsv'

import os
if not os.path.isfile(GAIA_PATH):

    # Set up the query to fetch stars with g-band magnitude limit
    query = f"""
    SELECT 
        source_id, ref_epoch,
        ra, dec, pmra, pmdec, 
        phot_g_mean_mag, 
        phot_bp_mean_mag, 
        phot_rp_mean_mag
    FROM gaiadr3.gaia_source
    WHERE phot_g_mean_mag < {MAGNITUDE_LIMIT}
    ORDER BY phot_g_mean_mag ASC
    """

    # Run the query asynchronously and dump to file
    job = Gaia.launch_job_async(query, dump_to_file=True, output_format='ecsv', output_file=GAIA_PATH+'.gz')
    os.rename(GAIA_PATH+'.gz', GAIA_PATH)  # already unzipped?

else:

    print("The query results already exist.")

## Construct Camera Model

In [ ]:
import sys
sys.path.append('../')
from STLib.sensor import Sensor 
from STLib.lens import Lens
from STLib.camera import Camera
from STLib.functions.psf import airyPSFModel, defocusPSFModel, pillboxPSFModel, gaussianPSFModel
from STLib.functions.dark import exponentialDarkModel
import STLib.utils

STLib.utils.ENABLE_TIMER = True

import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from astropy import units
from astropy.table import Table

In [ ]:
wavelength = 550  # nm
aperture = 50  # mm
focal_length = 30  # mm

In [ ]:
darkFunc = exponentialDarkModel(I0 = 0.5, T0 = 20, dT = 10)

# psf = airyPSFModel(wavelength=550*units.nm, aperture=aperture*units.mm, focal_length=focal_length*units.mm)
# psf = defocusPSFModel(wavelength=wavelength*units.nm, aperture=aperture*units.mm, focal_length=focal_length*units.mm, defocus=1.0*wavelength*units.nm)
# psf = pillboxPSFModel(radius=6*units.micron)
psf = gaussianPSFModel(sigma=10*units.micron)

In [ ]:
no_bloom = set()
full_bloom = {'+x','-x','+y','-y'}

sensor = Sensor(width_px=700, height_px=700, 
                px_len=6, px_pitch=6.5, 
                filter_efficiency=0.9, band='g', 
                quantum_efficiency=0.3, dark_current=darkFunc, 
                gain=5, full_well_capacity=1e6, 
                bloom=no_bloom, 
                readout_time=0, read_noise=25)

lens = Lens(aperture=aperture, focal_length=focal_length, 
            transmission_efficiency=0.99, 
            k1=0.4, k2=0.1, k4=0.1,
            p1=0.01,
            psf=psf, psf_bounds=(3,3))

camera = Camera(lens=lens, sensor=sensor, ra=0, dec=0, roll=0)

## Lens Distortion Visualization

In [ ]:
num_rows = 100
num_cols = 101
col_step = 10
row_step = 10
_x = np.linspace(0,1,num_cols)
_y = np.linspace(0,1,num_rows)
X,Y = np.meshgrid(_x,_y)
Xd, Yd = camera.lens.applyDistortion(X,Y)

fig, ax = plt.subplots()

# reference grid
for col in range(0, num_cols, col_step):
    ax.plot(X[:,col], Y[:,col], c='k', alpha=0.3, lw=1)
for row in range(0, num_rows, row_step):
    ax.plot(X[row,:], Y[row,:], c='k', alpha=0.3, lw=1)

# distorted grid
for col in range(0, num_cols, col_step):
    ax.plot(Xd[:,col], Yd[:,col], c='k', lw=1.2)
for row in range(0, num_rows, row_step):
    ax.plot(Xd[row,:], Yd[row,:], c='k', lw=1.2)
ax.set_aspect('equal')
ax.set_xlim([0,1])
ax.set_ylim([0,1])
plt.show()

## Dark Image

In [ ]:
camera.sensor.clear()
image = camera.snap(sky_mag=21.5, exposure_time=1.0, temperature=20, 
                    ra=[], 
                    dec=[], 
                    magnitudes=[])

In [ ]:
fig, ax = plt.subplots()
log_image = np.log10(image)
ax.imshow(log_image)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
plt.show()

## Gaia Image

In [ ]:
catalog_orig = Table.read(GAIA_PATH, format='ascii.ecsv')
catalog_orig = catalog_orig[catalog_orig['phot_g_mean_mag'] < 7]
print(f"Working with {len(catalog_orig)} stars.")

In [ ]:
camera.sensor.clear()
camera.orient(ra=40, dec=20, roll=45)
image = camera.snap(sky_mag=21.5, exposure_time=0.1, temperature=20, 
                    ra=catalog_orig['ra'].to(units.radian), 
                    dec=catalog_orig['dec'].to(units.radian), 
                    magnitudes=catalog_orig['phot_g_mean_mag'].value)

In [ ]:
fig, ax = plt.subplots()
log_image = np.log10(image)
ax.imshow(log_image, vmin=0)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
plt.show()

## Export Image

In [ ]:
from PIL import Image
image_uint8 = (255 * (log_image - log_image.min()) / log_image.max()).astype(np.uint8)
im = Image.fromarray(image_uint8, mode='L')
im.save('example-images/gaia-test.png')
display(im)